In [1]:
%pip install torch torchvision torchaudio


[notice] A new release of pip available: 22.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import spacy
from spacy.matcher import Matcher

passive_sentences = [
    "John was accused of committing crimes by David.",
    "She was sent a cheque for a thousand euros.",
    "He was given a book for his birthday.",
    "He will be sent away to school.",
    "The meeting was called off.",
    "He was looked after by his grandmother.",
]
active_sentences = [
    "David accused John of committing crimes.",
    "Someone sent her a cheque for a thousand euros.",
    "I gave him a book for his birthday.",
    "They will send him away to school.",
    "They called off the meeting.",
    "His grandmother looked after him."
]
composite_sentences = [
    "Three men seized me, and I was carried to the car."
]

# Load spaCy pipeline (model)
nlp = spacy.load('en_core_web_sm')
# Create pattern to match passive voice use
passive_rules = [
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'VBN'}],
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'VBZ'}],
    [{'DEP': 'nsubjpass'}, {'DEP': 'aux', 'OP': '*'}, {'DEP': 'auxpass'}, {'TAG': 'RB'}, {'TAG': 'VBN'}],
]
# Create pattern to match active voice use
active_rules = [
    [{'DEP': 'nsubj'}, {'TAG': 'VBD', 'DEP': 'ROOT'}],
    [{'DEP': 'nsubj'}, {'TAG': 'VBP'}, {'TAG': 'VBG', 'OP': '!'}],
    [{'DEP': 'nsubj'}, {'DEP': 'aux', 'OP': '*'}, {'TAG': 'VB'}],
    [{'DEP': 'nsubj'}, {'DEP': 'aux', 'OP': '*'}, {'TAG': 'VBG'}],
    [{'DEP': 'nsubj'}, {'TAG': 'RB', 'OP': '*'}, {'TAG': 'VBG'}],
    [{'DEP': 'nsubj'}, {'TAG': 'RB', 'OP': '*'}, {'TAG': 'VBZ'}],
    [{'DEP': 'nsubj'}, {'TAG': 'RB', 'OP': '+'}, {'TAG': 'VBD'}],
]

matcher = Matcher(nlp.vocab)  # Init. the matcher with a vocab (note matcher vocab must share same vocab with docs)
matcher.add('Passive',  passive_rules)  # Add passive rules to matcher
matcher.add('Active', active_rules)  # Add active rules to matcher
text = passive_sentences + active_sentences + composite_sentences  # Combine various passive/active sentences

for doc in nlp(" ".join(text)).sents:
    # doc = nlp(sentence)  # Process text with spaCy model
    matches = matcher(doc)  # Get matches
    # print("-"*40 + "\n" + sentence)
    if len(matches) > 0:
        for match_id, start, end in matches:
            string_id = nlp.vocab.strings[match_id]
            span = doc[start:end]  # the matched span
            print("\t{}: {}".format(string_id, span.text))
    else:
        print("\tNo active or passive voice detected.")


	Passive: John was accused
	Passive: She was sent
	Passive: He was given
	Passive: He will be sent
	Passive: meeting was called
	Passive: He was looked
	Active: David accused
	Active: Someone sent
	Active: I gave
	Active: They will send
	Active: They called
	Active: grandmother looked
	Active: men seized
	Passive: I was carried


In [9]:
import spacy
from spacy.matcher import Matcher

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

def get_sentence_tense(sentence):
    # Process the input sentence with spaCy
    doc = nlp(sentence)

    # Define the Matcher with a pattern for finding verbs with tense information
    matcher = Matcher(nlp.vocab)

    # Pattern for present tense (simple present or present continuous)
    present_pattern = [
        {"POS": "VERB", "dep": {"in": ["ROOT", "aux"]}, "tag": {"regex": "^VB.*"}},
    ]

    # Pattern for past tense
    past_pattern = [
        {"POS": "VERB", "dep": {"in": ["ROOT", "aux"]}, "tag": {"regex": "^VBD$"}},
    ]

    # Pattern for future tense
    future_pattern = [
        {"LOWER": "will"},
        {"POS": "VERB", "dep": {"in": ["ROOT", "aux"]}, "tag": {"regex": "^VB.*"}},
    ]

    matcher.add("PRESENT_PATTERN", [present_pattern])
    matcher.add("PAST_PATTERN", [past_pattern])
    matcher.add("FUTURE_PATTERN", [future_pattern])

    # Find matches in the parsed sentence
    matches = matcher(doc)

    # Determine the tense based on the matches
    if any(match_id == doc.vocab.strings["PRESENT_PATTERN"] for match_id, _, _ in matches):
        return "Present"
    elif any(match_id == doc.vocab.strings["PAST_PATTERN"] for match_id, _, _ in matches):
        return "Past"
    elif any(match_id == doc.vocab.strings["FUTURE_PATTERN"] for match_id, _, _ in matches):
        return "Future"
    else:
        return None

# Example usage
sentence = "I will go to the store."
tense = get_sentence_tense(sentence)

if tense:
    print(f"The tense of the sentence is: {tense}")
else:
    print("No relevant tense information found in the sentence.")


The tense of the sentence is: Present


In [29]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

# Define the patterns
past_tense_patterns = [
    [{'TAG': 'VBD'}],  # Simple past tense
    [{'TAG': 'VBN'}, {'DEP': 'auxpass'}],  # Past passive
    [{'LOWER': 'had'}, {'TAG': 'VBN'}],  # Past perfect
    [{'LOWER': 'had'}, {'LOWER': 'been'}, {'TAG': 'VBG'}]  # Past perfect continuous
]

present_tense_patterns = [
    [{'TAG': 'VBZ'}],  # 3rd person singular present
    [{'TAG': 'VBP'}],  # Non-3rd person singular present
    [{'TAG': 'VBG'}, {'DEP': 'aux'}],  # Present participle/gerund
    [{'LOWER': 'has'}, {'TAG': 'VBN'}],  # Present perfect
    [{'LOWER': 'has'}, {'LOWER': 'been'}, {'TAG': 'VBG'}]  # Present perfect continuous
]

future_tense_patterns = [
    [{'LOWER': 'will'}, {'TAG': 'VB'}],  # Future simple
    [{'LOWER': 'is'}, {'LOWER': 'going'}, {'LOWER': 'to'}, {'TAG': 'VB'}],  # Future with "going to"
    [{'LOWER': 'will'}, {'LOWER': 'be'}, {'TAG': 'VBG'}],  # Future continuous
    [{'LOWER': 'will'}, {'LOWER': 'have'}, {'TAG': 'VBN'}],  # Future perfect
    [{'LOWER': 'will'}, {'LOWER': 'have'}, {'LOWER': 'been'}, {'TAG': 'VBG'}]  # Future perfect continuous
]

# Add patterns to the matcher
matcher.add("PAST_TENSE", past_tense_patterns)
matcher.add("PRESENT_TENSE", present_tense_patterns)
matcher.add("FUTURE_TENSE", future_tense_patterns)

# Example sentences
sentences = [
    "I walked to the store.",
    "She is singing a song.",
    "We will go to the park.",
    "They had finished their homework.",
    "You are reading a book.",
    "I will be traveling next week.",
    "I am going to the store.",
    "You are not listening to me.",
    "I will have finished the homework by then.",
]

for sentence in sentences:
    doc = nlp(sentence)
    matches = matcher(doc)

    tense = None
    for match_id, start, end in matches:
        if nlp.vocab.strings[match_id] == "PAST_TENSE":
            tense = "Past"
        elif nlp.vocab.strings[match_id] == "PRESENT_TENSE":
            tense = "Present"
        elif nlp.vocab.strings[match_id] == "FUTURE_TENSE":
            tense = "Future"

    print(f'Sentence: "{sentence}" - Tense: {tense}')


Sentence: "I walked to the store." - Tense: Past
Sentence: "She is singing a song." - Tense: Present
Sentence: "We will go to the park." - Tense: Future
Sentence: "They had finished their homework." - Tense: Past
Sentence: "You are reading a book." - Tense: Present
Sentence: "I will be traveling next week." - Tense: Future
Sentence: "I am going to the store." - Tense: Present
Sentence: "You are not listening to me." - Tense: Present
Sentence: "I will have finished the homework by then." - Tense: Future
